<a href="https://colab.research.google.com/github/anaperez10/AnaliticaIII-Marketing/blob/main/Avances1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CARGA DE PAQUETES

In [3]:
#Importar paquetes
import pandas as pd
import numpy as np
import sqlite3 as sql
import plotly.graph_objs as go #Graficos
import plotly.express as px
from matplotlib.pyplot import figure
from sklearn import linear_model ## para regresión lineal
from sklearn import tree ###para ajustar arboles de decisión
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import RepeatedKFold
import multiprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.extmath import weighted_mode
weighted_mode
from sklearn.model_selection import cross_val_predict
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt ### gráficos
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
import os

IMPORTAR Y LEER BASES DE DATOS

In [9]:
#Ejecutar sql, conectar a bd y cargar tabla sql 
con = sql.connect('db_movies')
movies = pd.read_sql('select * from db_movies',con)

DatabaseError: ignored